Note: After the third run of Grid Search, some errors prompted so only the result for Method 2 remains... Anyway I will try to run it later. Fortunately I've stored some results from previous runs and I will use those results for my modelling.

In [1]:
import pandas as pd
from collections import Counter
import re
import numpy as np
from sklearn.utils import shuffle

# Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import f1_score, accuracy_score , recall_score , precision_score
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Natural Language Processing
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


E:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv('./news_dataset.csv',index_col='Unnamed: 0')

In [3]:
df=df.dropna()

In [4]:
df['fakeness']=df['label'].apply(lambda x:1 if x=='fake' else 0)

In [5]:
# Preparing the target and predictors for modeling

X_content_text = df.content.values
X_title_text = df.title.values
y = df.fakeness.values

In [8]:
# Stemming and Lemmatizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

from nltk.tokenize import RegexpTokenizer

Regexp = RegexpTokenizer(r'\w+')

tokenizer= Regexp.tokenize# Stemming and Lemmatizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

from nltk.tokenize import RegexpTokenizer

Regexp = RegexpTokenizer(r'\w+')

tokenizer= Regexp.tokenize

In [9]:
X_content_text_train, X_content_text_test, y_content_text_train, y_content_text_test = train_test_split(X_content_text,y, test_size = 0.2, random_state=1234,stratify=y)
X_title_text_train, X_title_text_test, y_title_text_train, y_title_text_test = train_test_split( X_title_text,y, test_size = 0.2, random_state=1234,stratify=y)

# Model 1: CountVectorizer & Logistic Regression 

In [ ]:
pipe = Pipeline([('cvec', CountVectorizer()),    
                 ('lr', LogisticRegression(solver='liblinear'))])

# Tune GridSearchCV
pipe_params = {'cvec__stop_words': [None, 'english'],
               'cvec__ngram_range': [(1,1), (2,2),(1,2),(1,3)],
               'cvec__tokenizer':[LemmaTokenizer(),Regexp.tokenize],
               'lr__C':[0.01, 0.1,1],
               'lr__penalty':['l1','l2']}

gs_content = GridSearchCV(pipe, param_grid=pipe_params, cv=3)

gs_content.fit(X_content_text_train, y_content_text_train);
print("Best score:", gs_content.best_score_)
print("Train score", gs_content.score(X_content_text_train, y_content_text_train))
print("Test score", gs_content.score(X_content_text_test, y_content_text_test))

gs_content.best_params_

In [ ]:
# get predictions for article section
y_content_text_pred = gs_content.predict(X_content_text_test)

# print metrics
print ("Logistig Regression F1 and Accuracy Scores : \n")
print ( "F1 score {:.4}%".format( f1_score(y_content_text_test, y_content_text_pred, average='macro')*100 ) )
print ( "Accuracy score {:.4}%".format(accuracy_score(y_content_text_test, y_content_text_pred)*100) )

# Model 2: CountVectorizer & MultinomialNB

In [ ]:
pipe = Pipeline([('cvec', CountVectorizer()),    
                 ('nb', MultinomialNB())])

# Tune GridSearchCV
pipe_params = {'cvec__stop_words': [None, 'english'],
               'cvec__ngram_range': [(1,1), (2,2),(1,2),(1,3)],
               'cvec__tokenizer':[LemmaTokenizer(),Regexp.tokenize],
               'nb__alpha': [.36, .6]}

gs_content = GridSearchCV(pipe, param_grid=pipe_params, cv=3)

gs_content.fit(X_content_text_train, y_content_text_train);
print("Best score:", gs_content.best_score_)
print("Train score", gs_content.score(X_content_text_train, y_content_text_train))
print("Test score", gs_content.score(X_content_text_test, y_content_text_test))

gs_content.best_params_


In [ ]:
# get predictions for article section
y_content_text_pred = gs_content.predict(X_content_text_test)

# print metrics
print ("Logistig Regression F1 and Accuracy Scores : \n")
print ( "F1 score {:.4}%".format( f1_score(y_content_text_test, y_content_text_pred, average='macro')*100 ) )
print ( "Accuracy score {:.4}%".format(accuracy_score(y_content_text_test, y_content_text_pred)*100) )

# Model 3： CountVectorizer & Random Forest

In [ ]:
pipe = Pipeline([('cvec', CountVectorizer()),    
                 ('rf', RandomForestClassifier(n_jobs=3))])

# Tune GridSearchCV
pipe_params = {'cvec__stop_words': [None, 'english'],
               'cvec__ngram_range': [(1,1), (2,2),(1,2),(1,3)],
               'cvec__tokenizer':[LemmaTokenizer(),Regexp.tokenize],
               'rf__n_estimators':[100,200,300],
               'rf__bootstrap': [True, False],
               'rf__max_features': ['auto', 'sqrt']}

gs_content = GridSearchCV(pipe, param_grid=pipe_params, cv=3)

gs_content.fit(X_content_text_train, y_content_text_train);
print("Best score:", gs_content.best_score_)
print("Train score", gs_content.score(X_content_text_train, y_content_text_train))
print("Test score", gs_content.score(X_content_text_test, y_content_text_test))

gs_content.best_params_

In [ ]:
# get predictions for article section
y_content_text_pred = gs_content.predict(X_content_text_test)

# print metrics
print ("Logistig Regression F1 and Accuracy Scores : \n")
print ( "F1 score {:.4}%".format( f1_score(y_content_text_test, y_content_text_pred, average='macro')*100 ) )
print ( "Accuracy score {:.4}%".format(accuracy_score(y_content_text_test, y_content_text_pred)*100) )

# Model 4: TF-IDF & Logistic Regression

In [ ]:
pipe = Pipeline([('tvect', TfidfVectorizer()),    
                 ('lr', LogisticRegression(solver='liblinear'))])

# Tune GridSearchCV
pipe_params = {'tvect__ngram_range': [(1,1),(1,2),(2,2),(1,3)],
               'tvect__max_df': [0.65,0.75,0.85,0.90],
               'tvect__min_df': [0.001,0.01],
               'tvect__min_df': [0.001,0.01],
               'tvect__tokenizer':[LemmaTokenizer(),Regexp.tokenize],
               'lr__C':[0.01, 0.1,1],
               'lr__penalty':['l1','l2']}

gs_content = GridSearchCV(pipe, param_grid=pipe_params, cv=3)

gs_content.fit(X_content_text_train, y_content_text_train);
print("Best score:", gs_content.best_score_)
print("Train score", gs_content.score(X_content_text_train, y_content_text_train))
print("Test score", gs_content.score(X_content_text_test, y_content_text_test))

gs_content.best_params_

In [ ]:
# get predictions for article section
y_content_text_pred = gs_content.predict(X_content_text_test)

# print metrics
print ("Logistig Regression F1 and Accuracy Scores : \n")
print ( "F1 score {:.4}%".format( f1_score(y_content_text_test, y_content_text_pred, average='macro')*100 ) )
print ( "Accuracy score {:.4}%".format(accuracy_score(y_content_text_test, y_content_text_pred)*100) )

# Model 5: TF-IDF & MultinomialNB¶

In [ ]:
pipe = Pipeline([('tvect', TfidfVectorizer()),    
                 ('nb', MultinomialNB())])

# Tune GridSearchCV
pipe_params = {'tvect__ngram_range': [(1,1),(1,2),(2,2),(1,3)],
               'tvect__max_df': [0.65,0.75,0.85,0.90],
               'tvect__min_df': [0.001,0.01],
               'tvect__min_df': [0.001,0.01],
               'tvect__tokenizer':[LemmaTokenizer(),Regexp.tokenize],
               'nb__alpha': [.36, .6]}

gs_content = GridSearchCV(pipe, param_grid=pipe_params, cv=3)

gs_content.fit(X_content_text_train, y_content_text_train);
print("Best score:", gs_content.best_score_)
print("Train score", gs_content.score(X_content_text_train, y_content_text_train))
print("Test score", gs_content.score(X_content_text_test, y_content_text_test))

gs_content.best_params_

In [ ]:
# get predictions for article section
y_content_text_pred = gs_content.predict(X_content_text_test)

# print metrics
print ("Logistig Regression F1 and Accuracy Scores : \n")
print ( "F1 score {:.4}%".format( f1_score(y_content_text_test, y_content_text_pred, average='macro')*100 ) )
print ( "Accuracy score {:.4}%".format(accuracy_score(y_content_text_test, y_content_text_pred)*100) )

# Model 6： TF-IDF & Random Forest

In [ ]:
pipe = Pipeline([('tvect', TfidfVectorizer()),    
                 ('rf', RandomForestClassifier(n_jobs=3))])

# Tune GridSearchCV
pipe_params = {'tvect__ngram_range': [(1,1),(1,2),(2,2),(1,3)],
               'tvect__max_df': [0.65,0.75,0.85,0.90],
               'tvect__min_df': [0.001,0.01],
               'tvect__min_df': [0.001,0.01],
               'tvect__tokenizer':[LemmaTokenizer(),Regexp.tokenize],
               'rf__n_estimators':[100,200,300],
               'rf__bootstrap': [True, False],
               'rf__max_features': ['auto', 'sqrt']}

gs_content = GridSearchCV(pipe, param_grid=pipe_params, cv=3)

gs_content.fit(X_content_text_train, y_content_text_train);
print("Best score:", gs_content.best_score_)
print("Train score", gs_content.score(X_content_text_train, y_content_text_train))
print("Test score", gs_content.score(X_content_text_test, y_content_text_test))

gs_content.best_params_

In [ ]:
# get predictions for article section
y_content_text_pred = gs_content.predict(X_content_text_test)

# print metrics
print ("Logistig Regression F1 and Accuracy Scores : \n")
print ( "F1 score {:.4}%".format( f1_score(y_content_text_test, y_content_text_pred, average='macro')*100 ) )
print ( "Accuracy score {:.4}%".format(accuracy_score(y_content_text_test, y_content_text_pred)*100) )